# Noise‑Refinement Workflow
1. Laden der Daten
2. Nachklassifikation via Cosine‑Ähnlichkeit zu bestehenden Clusterzentren
3. Analyse der Reassignment‑Ergebnisse
4. Re‑Clustering des verbleibenden Noise‑Kerns mit HDBSCAN
5. Visualisierung mit UMAP
6. Speichern des finalen Labels



In [1]:
%pip install --quiet umap-learn hdbscan joblib

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import joblib, hdbscan, umap.umap_ as umap
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (8,6)
print('Libraries geladen.')

Libraries geladen.


## 1 | Daten laden

In [3]:
noise_df = pd.read_pickle('noise_questions.pkl')
centers = joblib.load('cluster_centers.pkl')
print(f'Noise‑Fragen: {len(noise_df)} | Clusterzentren: {len(centers)}')
noise_df.drop(columns=['embedding']).head()


Noise‑Fragen: 1890 | Clusterzentren: 209


,ID_Wahl,Datum,Frage_ID,Frage_Text,Frage_Typ,Bereich_ID,Bereich,ID_gesamt,Sprache,text_norm,dup_grp,is_rep,cluster
0,2.0,2019-10-20 00:00:00,5.0,"Wie beurteilen Sie diese Aussage: ""Wer sich ni...",options7,4734.0,Werthaltungen,Q00000,de,wie beurteilen sie diese aussage wer sich nich...,0,True,-1
1,2.0,2019-10-20 00:00:00,8.0,"Wie beurteilen Sie diese Aussage: ""Die Bestraf...",options7,4734.0,Werthaltungen,Q00001,de,wie beurteilen sie diese aussage die bestrafun...,1,True,-1
2,2.0,2019-10-20 00:00:00,11.0,"Wie beurteilen Sie diese Aussage: ""Für ein Kin...",options7,4734.0,Werthaltungen,Q00002,de,wie beurteilen sie diese aussage für ein kind ...,2,True,-1
10,2.0,2019-10-20 00:00:00,47.0,Sollen die Renten der Pensionskasse durch eine...,options4,4160.0,Sozialstaat & Familie,Q00010,de,sollen die renten der pensionskasse durch eine...,10,True,-1
11,2.0,2019-10-20 00:00:00,50.0,Befürworten Sie Bestrebungen in den Kantonen z...,options4,4160.0,Sozialstaat & Familie,Q00011,de,befürworten sie bestrebungen in den kantonen z...,11,True,-1


## 2 | Nachklassifikation an bestehende Cluster

In [4]:
sims = cosine_similarity(np.vstack(noise_df['embedding']), np.vstack(centers.values))
best_cluster = sims.argmax(axis=1)
best_score = sims.max(axis=1)

thresh = 0.7  # frei anpassen, 0.30 ≈ 70 % Ähnlichkeit
mask_reassign = best_score >= thresh
noise_df.loc[mask_reassign, 'reassign_to'] = [list(centers.index)[i] for i in best_cluster[mask_reassign]]
noise_df['reassign_score'] = best_score
print('Reassignments:', mask_reassign.sum())

Reassignments: 1165


### 2.1 | Übersicht der Zuordnung

In [5]:
noise_df['reassign_to'].value_counts(dropna=False).head()

reassign_to
NaN      725
205.0     36
175.0     25
157.0     23
122.0     23
Name: count, dtype: int64

## 3 | Verbleibender Noise‑Kern

In [6]:
remain_df = noise_df[noise_df['reassign_to'].isna()].copy()
print('Verbleibender Noise:', len(remain_df))

Verbleibender Noise: 725


## 4 | Re‑Clustering mit HDBSCAN

In [7]:
hdb = hdbscan.HDBSCAN(min_cluster_size=2, metric='euclidean')
remain_df['subcluster'] = hdb.fit_predict(np.vstack(remain_df['embedding']))
remain_df['subcluster'].value_counts().head()

c:\Users\olive\git\EPRP\.conda\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\olive\git\EPRP\.conda\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


subcluster
 2    523
-1    194
 1      3
 0      3
 3      2
Name: count, dtype: int64

## 5 | Ergebnisse speichern

In [8]:
final_df = pd.concat([
    noise_df[mask_reassign],
    remain_df
])
final_df.to_pickle('noise_refined.pkl')
print('Gespeichert unter noise_refined.pkl')

Gespeichert unter noise_refined.pkl


## Analyse der neu zugeordneten Fragen

In [9]:
# 1 | Daten laden und vorbereiten
import pandas as pd, numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import joblib

# Hauptdaten
rep_df   = pd.read_pickle("rep_df.pkl")          # Haupt-DF mit allen Original-Clustern
noise_df = pd.read_pickle("noise_refined.pkl")   # dein exportiertes Noise-DF

# Zentren korrekt laden
centers  = joblib.load("cluster_centers.pkl")      # oder joblib.load(...)



In [10]:
# 2 | Hauptfrage (Repräsentant) pro Cluster bestimmen
# 2.1  Hilfsfunktion
def cluster_representative(label):
    emb_center = centers[label]
    # alle Originalfragen dieses Clusters
    cand = rep_df[rep_df["cluster"] == label]
    sims = cosine_similarity(np.vstack(cand["embedding"]), [emb_center]).ravel()
    best_idx = sims.argmax()
    return cand.iloc[best_idx]["Frage_Text"]

# 2.2  Dictionary anlegen
repr_dict = {lbl: cluster_representative(lbl) for lbl in centers.index}


In [11]:
# 3 | DataFrame der neu zugeordneten Fragen bauen
# 3.1  Nur Fragen, die tatsächlich reassigned wurden
reassigned_df = noise_df[noise_df["reassign_to"].notna()].copy()

# 3.2  Hauptfrage dazumerken
reassigned_df["cluster_main_question"] = reassigned_df["reassign_to"].map(repr_dict)

# 3.3  Optional sortieren nach Zielcluster
reassigned_df.sort_values("reassign_to", inplace=True)


In [12]:
# 4 | Ergebnisse anschauen

pd.set_option('display.max_colwidth', None)

# 4.1 Schnell-Übersicht
print(reassigned_df[["Frage_Text", "reassign_to", "cluster_main_question"]].head(10))


# 4.2 Für einen bestimmten Cluster

CLUSTER_ID = 7
reassigned_df[reassigned_df["reassign_to"] == CLUSTER_ID]   \
    [["Frage_Text", "cluster_main_question"]]


                                                                                                                                            Frage_Text  \
6459                                                                                                                                       Sozialwerke   
3833                                                                                                                                Soziale Sicherheit   
3027        Sollen Sozialhilfebeziehende weiterhin zu einem Vorbezug aus der Pensionskasse verpflichtet werden können (vor dem Bezug von Sozialhilfe)?   
1185                                                                          Soll das Land im Bereich «Soziale Wohlfahrt» mehr oder weniger ausgeben?   
71                                                                            Soll der Bund im Bereich "Soziale Wohlfahrt" mehr oder weniger ausgeben?   
69                                                                       Sol

,Frage_Text,cluster_main_question
3559,Städtische Angestellte können in Bern heute mit 63 Jahren in Rente gehen. Soll das ordentliche Rentenalter auf die üblichen 65 Jahre erhöht werden?,Befürworten Sie eine Erhöhung des Rentenalters für Frauen und Männer (z.B. auf 67 Jahre)?
3656,"Sind Sie dafür, das ordentliche Renteneintrittsalter an die durchschnittliche Lebenserwartung anzupassen?",Befürworten Sie eine Erhöhung des Rentenalters für Frauen und Männer (z.B. auf 67 Jahre)?
